<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/CNN_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra # เป็นการ import numpy เพื่อเปิดใช้ฟังก์ชันในการทำงาน
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) # เป็น library ในการจัดการ dataframe
import tensorflow as tf #library สำหรับสร้าง machine learning models แบบ open source จาก Google
import datetime,os # datetime ใช้ในการจัด format ของวัน-เวลาให้อยู่ในรูปแบบที่ต้องการ 
import math # เรียกใช้ module เพื่อคำนวณค่าต่างๆ 
import matplotlib.pyplot as plt # เป็น module พื้นฐานที่ใช้ในการสร้างกราฟของ python
import seaborn as sns # เป็นฟังก์ชันเสริมที่ใช้สร้างกราฟให้มีความสวยงามมากขึ้น
from sklearn.model_selection import train_test_split # เป็น library ในการพัฒนาโปรแกรมโดยใช้การเรียนรู้ของเครื่อง

In [2]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [4]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [5]:
#loading dataframes # เอาที่อยู่ของไฟล์เชื่อมกับที่อยู่ของไฟล์เรา โดยกำหนดชื่่อไฟล์ที่เราต้องการ
train_path = os.path.join(path,'train.csv')
train_df = pd.read_csv(train_path)
val_path = os.path.join(path,'validation.csv')
val_df = pd.read_csv(val_path)
test_path = os.path.join(path,'boneage-test-dataset.csv')
test_df = pd.read_csv(test_path)

In [6]:
#appending file extension to id column for both training and testing dataframes
# ต่อท้าย data ใน column 'id' ด้วย .png
train_df['id'] = train_df['id'].apply(lambda x: str(x)+'.png') # ทำการเปลี่ยน type ใน column 'id' จาก int  เป็น string เพื่อให้สามารถต่อท้ายด้วย .png ได้ # ฟังก์ชันแลมบ์ดาเป็นฟังก์ชันที่ไม่ระบุตัวตนขนาดเล็ก
val_df['id'] = val_df['id'].apply(lambda x: str(x)+'.png') 
test_df['Case ID'] = test_df['Case ID'].apply(lambda x: str(x)+'.png') 

train_df.head() # แสดงค่าของ train_df บางส่วน

,id,boneage,male
0,1377.png,180,False
1,1378.png,12,False
2,1379.png,94,False
3,1380.png,120,True
4,1381.png,82,False


In [7]:
#finding out the number of male and female children in the dataset
#creating a new column called gender to keep the gender of the child as a string
train_df['gender'] = train_df['male'].apply(lambda x: 'male' if x else 'female') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
val_df['gender'] = val_df['male'].apply(lambda x: 'male' if x else 'female') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 


In [8]:
mean_bone_age_train = train_df['boneage'].mean() 
std_bone_age_train = train_df['boneage'].std()
train_df['bone_age_z'] = (train_df['boneage'] - mean_bone_age_train)/(std_bone_age_train)
print(train_df.head())

         id  boneage   male  gender  bone_age_z
0  1377.png      180  False  female    1.278306
1  1378.png       12  False  female   -2.796338
2  1379.png       94  False  female   -0.807524
3  1380.png      120   True    male   -0.176924
4  1381.png       82  False  female   -1.098570


In [9]:
mean_bone_age_val = val_df['boneage'].mean() 
std_bone_age_val = val_df['boneage'].std()
val_df['bone_age_z'] = (val_df['boneage'] - mean_bone_age_val)/(std_bone_age_val)
print(val_df.head())

          id  boneage   male  gender  bone_age_z
0  10186.png       10   True    male   -2.864366
1  10758.png       15  False  female   -2.742400
2  10860.png      180  False  female    1.282468
3  10877.png      180  False  female    1.282468
4  11008.png      180  False  female    1.282468


In [10]:
#import matplotlib.image as mpimg #นำเข้า library ที่ใช้อ่านรูป
#for filename, boneage, gender in train_df[['id','boneage','gender']].sample(10).values: #เป็นคำสั่งวนลูปเพื่อแสดงภาพ และข้อมูลของภาพที่เก็บไว้ใน Train_df (ชื่อ อายุ เพศ)
#    img = mpimg.imread(os.path.join(path,'boneage-training-dataset-5261/')+ filename) # เป็นคำสั่งที่ใช้ในการนำรูปเข้ามา
#    plt.imshow(img) # แสดงรูปภาพ
#    plt.title('Image name:{}  Bone age: {} years  Gender: {}'.format(filename, boneage/12, gender)) # แสดงหัวเรื่องโดยบอกถึง ชื่อ อายุ(ปี) และเพศ ของแต่ละภาพ
#    plt.axis('off')
#    plt.show()

In [11]:
#library required for image preprocessing 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from  keras.applications.xception import preprocess_input

In [12]:
#reducing down the size of the image 
img_size = 256 # ลดขนาดรูปภาพ

In [13]:
train_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
val_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

In [14]:
#train data generator
train_generator = train_data_generator.flow_from_dataframe(
    dataframe = train_df,
     directory = (os.path.join(path,'boneage_training_dataset/')),
    x_col= 'id',
    y_col= 'bone_age_z',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode= 'raw',
    flip_vertical = True,
    color_mode = 'rgb',
    target_size = (img_size, img_size))

Found 10089 validated image filenames.


In [29]:
#validation data generator
val_generator = val_data_generator.flow_from_dataframe(
    dataframe = val_df,
    directory = (os.path.join(path,'boneage_validation_dataset/')),
    x_col = 'id',
    y_col = 'bone_age_z',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'raw',
    flip_vertical = True,
    color_mode = 'rgb',
    target_size = (img_size, img_size))

Found 2522 validated image filenames.


In [16]:
#test data generator
test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_data_generator.flow_from_directory(
    directory = (os.path.join(path,'boneage-test-dataset/')),
    target_size = (img_size,img_size),
    shuffle = True,
    class_mode = 'binary',classes=['.'],
    color_mode = 'rgb')

Found 200 images belonging to 1 classes.


In [17]:
test_X, test_Y = next(val_data_generator.flow_from_dataframe( 
                            val_df, 
                            directory = (os.path.join(path,'boneage_validation_dataset/')),
                            x_col = 'id',
                            y_col = 'bone_age_z', 
                            target_size = (img_size, img_size),
                            batch_size = 2523,
                            class_mode = 'raw'
                            )) 

Found 2522 validated image filenames.


#Model

In [18]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [19]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [35]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [20]:
# Define relevant variables for the ML task
START_EPOCH=0
NUM_EPOCHS=60
LR=0.0001
PATIENCE=2
BATCH_SIZE=20
NUM_WORKERS=0
NUM_GPUS=1
GPUS=0

In [22]:
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
import torchvision.datasets as datasets
from torch.autograd import Variable

In [24]:
# Dataloaders
train_transform = transforms.Compose([transforms.Resize((256, 256)),
                               transforms.RandomAffine(
                                   20, translate=(0.2, 0.2),
                                   scale=(1, 1.2)),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor()])

val_transform = transforms.Compose([transforms.Resize((256, 256)),
                               transforms.ToTensor()])

In [25]:
# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_generator,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)

train_loader

In [26]:
val_loader = torch.utils.data.DataLoader(dataset = val_generator,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)
val_loader

In [27]:
test_loader = torch.utils.data.DataLoader(dataset = test_generator,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)
test_loader

In [30]:
Sample_X_data = Variable(torch.Tensor(test_X))

In [31]:
Sample_X_data

tensor([[[[-0.9059, -0.9059, -0.9059],
          [-0.9137, -0.9137, -0.9137],
          [-0.8980, -0.8980, -0.8980],
          ...,
          [-0.8902, -0.8902, -0.8902],
          [-0.8902, -0.8902, -0.8902],
          [-0.8745, -0.8745, -0.8745]],

         [[-0.9059, -0.9059, -0.9059],
          [-0.8980, -0.8980, -0.8980],
          [-0.9059, -0.9059, -0.9059],
          ...,
          [-0.8824, -0.8824, -0.8824],
          [-0.8745, -0.8745, -0.8745],
          [-0.8902, -0.8902, -0.8902]],

         [[-0.9059, -0.9059, -0.9059],
          [-0.8980, -0.8980, -0.8980],
          [-0.9059, -0.9059, -0.9059],
          ...,
          [-0.8745, -0.8745, -0.8745],
          [-0.8824, -0.8824, -0.8824],
          [-0.8824, -0.8824, -0.8824]],

         ...,

         [[-0.7961, -0.7961, -0.7961],
          [-0.7961, -0.7961, -0.7961],
          [-0.8039, -0.8039, -0.8039],
          ...,
          [-0.7961, -0.7961, -0.7961],
          [-0.7961, -0.7961, -0.7961],
          [-0.7882, -0

In [32]:
Sample_Y_data = Variable(torch.Tensor(test_Y))

In [33]:
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.layer = torch.nn.Linear(1, 1)

   def forward(self, x):
       x = self.layer(x)      
       return x

net = Net()
print(net)

Net(
  (layer): Linear(in_features=1, out_features=1, bias=True)
)


In [34]:
# Define Optimizer and Loss Function
optimizer = torch.optim.SGD(net.parameters(), lr=LR)
loss_func = torch.nn.MSELoss()

In [36]:
model = Net()
criterion = nn.L1Loss()

#Training

In [37]:
inputs = Variable(Sample_X_data)
outputs = Variable(Sample_Y_data)
for i in range(60):
   prediction = Net(inputs)
   loss = loss_func(prediction, outputs) 
   optimizer.zero_grad()
   loss.backward()        
   optimizer.step()       

   if i % 10 == 0:
       # plot and show learning process
       plt.cla()
       plt.scatter(Sample_X_data.data.numpy(), Sample_Y_data.data.numpy())
       plt.plot(Sample_X_data.data.numpy(), prediction.data.numpy(), 'r-', lw=2)
       plt.text(0.5, 0, 'Loss=%.4f' % loss.data.numpy(), fontdict={'size': 10, 'color':  'red'})
       plt.pause(0.1)

plt.show()

TypeError: ignored